## 파라메터 서칭
tree base 모델은 설정 가능한 파라메터의 조합에 따라 모델 예측력 차이가 큰 특징을 가지고 있습니다.  
특히, Xgboost 모델의 경우 파라메터 설정에 따른 모델 예측력 차이가 굉장히 크기에 꼭 파라메터 서칭을 진행해주셔야 합니다.  
간단한 문법을 통해 파라메터 서칭을 진행 해보겠습니다.

In [ ]:
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

from itertools import product

# product 함수로 파라메터의 모든 조합 만들기
# 서칭할 파라메터의 조합
depth = [12, 13, 14] # 서칭할 파라메터 레인지 설정
est = [60, 70, 80, 100]

best_rmse2 = 999999999999999 # 최소값을 찾기 위한 변수 초기화
best_param2 = None

for param in list(product(depth, est)): # product 로 파라메터 레인지의 모든 조합 생성 
    print(f'파라메터 서칭 중 {param[0]} depth, {param[1]} est 서칭 중')
    # 조합을 순환하면서 모델 생성 후 학습
    model = RandomForestRegressor(random_state=42, max_depth=param[0], n_estimators=param[1])
    model.fit(X_train2, y_train2)
    pred = model.predict(X_val) # 평가를 위한 val 사용
    rmse = mean_squared_error(y_val, pred, squared=False)
    if rmse < best_rmse2: # 최소값 찾는 알고리즘에 따라 최적 모델의 rmse와 param 저장
        best_rmse2 = rmse
        best_param2 = param
print('학습완료!')

In [ ]:
# 위 파라메터 서칭 코드로 찾은 최적 모델의 평가지표와 파라메터 확인
best_rmse2, best_param2

In [ ]:
# 최적 모델로 모델 다시 학습 및 평가
best_model = RandomForestRegressor(random_state=42, max_depth=best_param[0], n_estimators=best_param[1])
best_model.fit(X_train, y_train)
best_pred = best_model.predict(X_test)
print(r2_score(y_test, best_pred))
print(mean_squared_error(y_test, best_pred, squared=False))

### sklearn GridSearchCV
sklearn 패키지에는 위의 파라메터 서칭 과정을 간편하게 진행 할 수 있도록 GridSearchCV 방법론을 제공합니다.  
기존 파라메터 서칭과 함께 cross validation 과정을 추가하여 데이터 분할에 강건한 모델을 선택할 수 있도록 제작 되었습니다.

In [ ]:
''' xgboost 파라메터 서칭 목록
max_depth : classification [3, 5, 7, 9], regression [7, 9, 11, 13, 15]
n_estimators : 부스팅 트리 갯수, [100, 300, 500, 700, 1000, 2000, 3000]
learning_rate : round별 학습률 [0.001, 0.003, 0.01, 0.03]
subsample : 부트스트랩 샘플 비율 [0.6, 0.7, 0.8]
colsample_bytree : 부트스트랩 컬럼 비율 [0.6, 0.7, 0.8]
설명변수 갯수가 많은 경우 아래 파라메터도 서칭
reg_alpha : L1, lasso [1, 3, 5, 7, 9]
reg_lambda : L2, ridge, [1, 3, 5, 7, 9]
'''

In [ ]:
# 그리드 서치 import
from sklearn.model_selection import GridSearchCV

In [ ]:
# 그리드 서치를 위한 모델 및 파라메터 준비
model = XGBRegressor(random_state=42, verbosity=1)
params = {
    'max_depth':[7, 9, 11, 13, 15],
    'n_estimators': [100, 300, 500, 700],
    'learning_rate': [0.001, 0.003, 0.01, 0.03],
    'subsample' : [0.6, 0.7, 0.8],
    'colsample_bytree' : [0.6, 0.7, 0.8]
}

In [ ]:
# 그리드 서치 실습
grid = GridSearchCV(estimator=model, param_grid=params, scoring='r2', verbose=1, cv=5)
'''
estimator : 모델 딕셔너리
param_grid : 파라메터 딕셔너리
scoring=None : 평가방법
n_jobs=None : 학습에 사용할 컴퓨터 코어 갯수
verbose=0 : 리포트 형식 0, 1, 2

scoring 참고
https://scikit-learn.org/stable/modules/model_evaluation.html
'''

In [ ]:
# grid 학습
grid.fit(X_train, y_train)

In [ ]:
# 최적 모델 및 파라메터 확인
grid.best_score_, grid.best_params_

In [ ]:
# 최적 모델로 모델 다시 학습 및 평가
best_model = XGBRegressor(random_state=42, 
                          max_depth=grid.best_params_['max_depth'],
                          n_estimators=500,
                          learning_rate=0.03,
                          subsample=0.8,
                          colsample_bytree=0.8)
best_model.fit(X_train, y_train)
best_pred = best_model.predict(X_test)
print(r2_score(y_test, best_pred))
print(mean_squared_error(y_test, best_pred, squared=False))

In [ ]:
# 분석 프로세스 중 가장 간편한 부분
# 컴퓨터 스펙에 따라 시간 단축이 가능한 부분 파라메터 서칭도 자동화 가능
# 모델 서칭 (데이터를 사용가능한 모델을 모두 피팅)
# 모델 선택 후 여러번의 파라메터 서칭 진행
# 여러번의 파라메터서칭을 진행하는 과정을 거칩니다.